<a href="https://colab.research.google.com/github/mukund-rajukumar/My-VisionAI-Projects/blob/main/FoodAndBevPackageDetection/FnB_CPG_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimizing Vending Machine Inventory with AI: A Deep Dive into Model Selection

### Introduction:
Vending machines, once simple dispensers, are evolving into data-driven hubs. To maximize customer satisfaction and revenue, owners need real-time insights into product consumption. This article explores how Vision AI can enhance inventory management by using cameras to monitor purchases.

### The Challenge:
Traditional inventory methods often rely on manual checks or unreliable sensors. Vision AI offers a more accurate and efficient solution. However, the diversity of products in vending machines, coupled with varying package sizes and shapes, presents a unique challenge.

### The Solution: Deep Learning
Deep learning, a subset of AI, is ideally suited for this task. It can learn complex patterns from vast datasets, making it capable of identifying and classifying products even in the face of variations.

### Beyond the model: The Vision AI stack
While the AI model is crucial, it is part of the larger vision AI solution stack. Key components include:
1. Hardware Selection: Choosing the right camera with desired FoV that can capture high quality images in various lighting conditions
2. Hardware Installation: Ensuring cameras are securely mounted and positioned to capture optimal product views without being intrusive or compromising customer experience.
3. Data Transmission: Selecting reliable communication mechanism to transmit image/video data to the processing system
4. Processing System: Determining whether to run the AI model on-device (edge computing) or in the cloud or in a hybrid fashion.
5. Data Storage: Saving necessary data to support future analysis and reporting
6. Data Presentation: Helping owners visualize the insights via a front-end application

### Model Development: A Deep Dive

To accurately track product purchases, we need a reference database containing images and corresponding information for each product in the vending machine, including product name and location within the machine. Our motion-detection camera captures footage whenever a customer makes a purchase. By analyzing the movement of the product from its slot, we can effectively identify the specific item purchased.

Our AI models must perform the following tasks:
1. Object Detection: Identify and locate individual products within a vending machine image by placing bounding boxes around them.
2. Optical Character Recognition (OCR): Extract text from the cropped images within each bounding box to determine the product name. These names will serve as the classes or product types carried by the vending machine.
3. Object Classification (Optional): Given an image of a product, accurately categorize it into one of the predefined classes based on its visual features. Alternatively, we can use the images captured before and after a purchase to identify the slot/bounding box from which the product was taken. This information can then be mapped back to the product name using the reference database.

An AI model development involved the following 6 steps:
1. Data Collection: Gather a diverse dataset of images depicting various products in vending machines. Ensure the dataset covers different lighting conditions, angles, and product placements.
2. Data Preprocessing: Clean and prepare the data for training. This may involve resizing images, normalizing pixel values, and augmenting the dataset to introduce variations.
3. Model Architecture Selection: Choose a suitable deep learning architecture, such as a convolutional neural network (CNN), which excels at image recognition tasks.
4. Training: Train the model on the prepared dataset. This involves feeding the model images and their corresponding labels, allowing it to learn the underlying patterns.
5. Validation: Evaluate the model's performance on a separate validation dataset to assess its generalization ability.
6. Inference: Deploy the trained model to analyze real-time camera feeds and predict the products being purchased.

We will look into each one of these 6 steps in detail for both Object Detection model and the OCR model, starting with Object Detection model first.

## Object Detection Model
In order to determine whether I can use an existing pre-trained AI model as-is for my application or will I have to train or fine-tune a model, I had to evaluate whether any of the pre-trained models will work or not. So, I shortlisted top object detection models:
Many object detectors use 2 stages, first generating object proposals to identify regions of interest in an image using one AI model and then passing these regions to a detection AI model to classify them. One stage methods prioritize inference speed over detection accuracy. Below are the top 4 detector models:

Single stage detectors
1. YOLO (You only look once)
2. SSD (Single Shot Detector)

Two-stage detectors
3. Detectron
4. Faster R-CNN  

Let's quickly evaluate how these pre-trained versions of the above models perform for this application that requires detecting packaged food and beverages.